# Créer les données

## Générer les product_ids et prix de manière cohérente

In [186]:
import random
import datetime
import pandas as pd

# Avec une dict-comprehension
product_ids = list(range(1, 113))
price_per_id = {
    product_id: round(random.uniform(1.15, 100.45), 2)
    for product_id in product_ids
}


# _id = 1
# price_per_id = {}
# while _id <= 112:
#    price_per_id[_id] = round(random.uniform(1.15, 100.45), 2)
#    _id = _id + 1

first_day = datetime.datetime(
        year=2023, 
        month=1, 
        day=1
    )
last_day = datetime.datetime(
        year=2023, 
        month=1, 
        day=31,
        hour=23,
        minute=59,
        second=59
    )

seconds_in_between = (last_day - first_day).total_seconds()

records = []
for trial in range(10_000):
    transaction_id = trial
    product_id = random.choice(product_ids)
    unit_price = price_per_id[product_id]
    quantity = random.randint(1, 100) # le dernier est bien inclus
    
    datetime_ = first_day + datetime.timedelta(
        seconds=random.randint(0, seconds_in_between)
        )
    
    record = {
     #   "id": transaction_id,
        "product_id":product_id,
        "unit_price":unit_price,
        "quantity":quantity,
        "datetime":datetime_,
    }
    records.append(record)
    
    
import sqlite3
 
# Connecting to sqlite
# connection object
conn = sqlite3.connect('my.db')

c = conn.cursor()
c.execute("DROP TABLE IF EXISTS transactions;")

create_table = """CREATE TABLE IF NOT EXISTS transactions (
                    id integer PRIMARY KEY AUTOINCREMENT,
                    product_id integer,
                    unit_price FLOAT,
                    quantity integer,
                    datetime DATETIME
                ); """

c.execute(create_table)


# Insertion des enregistrements dans la table transaction
for record in records:
    row = (record['product_id'], record['quantity'], record['unit_price'], str(record['datetime']))
    c.execute("INSERT INTO 'transactions' (product_id, quantity, unit_price, datetime) VALUES (?, ?, ?, ?)", row)
    conn.commit()

conn.close()

In [268]:
conn = sqlite3.connect('my.db')

class TransactionManager:
    def __init__(self, conn):
        self.conn = conn
    
    def read(self, _id:int):
        
        if not isinstance(_id, int):
            raise TypeError(f'{_id} is not an integer.')
        
        sql = "SELECT * FROM transactions WHERE id = ?"
        c = self.conn.cursor()
        c.execute(sql, (_id,))
        item = c.fetchone()
        
        if item is None:
            raise ValueError(f'{_id} is not an existing id in transactions table.')
        
        return item
    

    def create(self, record):
        sql = "INSERT INTO 'transactions' (product_id, quantity, unit_price, datetime) VALUES (?, ?, ?, ?)"
        values = ( record['product_id'], record['quantity'], record['unit_price'], str(record['datetime']))
        c = self.conn.cursor()
        c.execute(sql, values)
        self.conn.commit()
        
        return cursor.lastrowid
    
    
    def update(self, _id, partial_record):
        block_set = " , ".join(
            [
               f"{key} =  ?" for key in partial_record
            ]
        )
                    
        sql = f'''UPDATE transactions SET {block_set} WHERE id = {_id}'''        
        # Même ordre que dans block_set : (nom des colonnes et valeurs)
        values = tuple(partial_record.get(key) for key in partial_record)

        c = self.conn.cursor()
        c.execute(sql, values)
        self.conn.commit()
        
        return _id
    

In [203]:

tm = TransactionManager(conn=conn)


row = tm.read(123)


# Création d'un enregistrement
record = {
    'product_id': 42,
    'quantity': 5,
    'unit_price': 10.99,
    'datetime': '2023-05-30 12:00:00'
}
created_id = tm.create(record)
    
created_id


tm.update(
    1110,
    { 
        "datetime":'2023-08-30 12:00:00' ,
        "quantity": 28,
        "unit_price": 17.21
    }
)

10000

10000

In [80]:
cur.execute(
    """INSERT INTO transactions (product_id, unit_price, quantity, datetime)
     VALUES (:product_id, :unit_price, :quantity, :datetime);""", 
    records[0]
)

OperationalError: table transactions has no column named product_id

In [ ]:
insert_into = ''' INSERT INTO transactions(product_id , unit_price, quantity, datetime)
          VALUES(?,?,?,?) '''
cur = conn.cursor()
cur.execute(insert_into, records[0])
conn.commit()

In [67]:
import sqlite3
 
# Connecting to sqlite
# connection object
connection_obj = sqlite3.connect('geek.db')
 
# cursor object
cursor_obj = connection_obj.cursor()
 
# Drop the GEEK table if already exists.
cursor_obj.execute("DROP TABLE IF EXISTS transactions;")